Nama: Putri Fatiha Nuzula

NIM: 2602193042

Kelas: LC09

In [ ]:
import pandas as pd
import numpy as np

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 2.a

In [ ]:
df = pd.read_csv('data_1D.csv')

In [ ]:
df.head()

,Unnamed: 0,text,label
0,0,Tamron AF 70-300mm F/4.0-5.6 Di LD Macro Telep...,Electronics
1,1,Softline Butterfly Women's Cotton Camisole,Clothing & Accessories
2,2,Indian Economy: For UPSC Civil Services & Othe...,Books
3,3,Fujifilm X-T3 Mirrorless Digital Camera with X...,Electronics
4,4,"Barata Formal Ties For Men, Navy Blue Tie Form...",Clothing & Accessories


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12606 entries, 0 to 12605
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  12606 non-null  int64 
 1   text        12606 non-null  object
 2   label       12606 non-null  object
dtypes: int64(1), object(2)
memory usage: 295.6+ KB


In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True) # drop unrelated column

In [ ]:
# label encode for 'label' column
from sklearn import preprocessing
label_enc = preprocessing.LabelEncoder()

df['label']= label_enc.fit_transform(df['label'])

In [ ]:
df.head()

,text,label
0,Tamron AF 70-300mm F/4.0-5.6 Di LD Macro Telep...,2
1,Softline Butterfly Women's Cotton Camisole,1
2,Indian Economy: For UPSC Civil Services & Othe...,0
3,Fujifilm X-T3 Mirrorless Digital Camera with X...,2
4,"Barata Formal Ties For Men, Navy Blue Tie Form...",1


In [ ]:
df['label'].value_counts()

,count
label,
3,4839
0,2962
2,2615
1,2190


In [ ]:
def cleansing(df):
    df_clean=df.str.lower() # convert to lowercase
    df_clean=[re.sub(r"\d+","",i )for i in df_clean] # remove numbers
    df_clean=[re.sub(r'[^\w]', ' ', i)for i in df_clean] # remove punctuation
    df_clean=[re.sub(r'\s+',' ',i)for i in df_clean] # remove multiple whitespaces
    df_clean=[re.sub(r'[^\x00-\x7F]+', '',i)for i in df_clean] # remove non-ASCII characters
    return df_clean

In [ ]:
df['clean_text']=cleansing(df['text'])
df.head()

,text,label,clean_text
0,Tamron AF 70-300mm F/4.0-5.6 Di LD Macro Telep...,2,tamron af mm f di ld macro telephoto zoom lens...
1,Softline Butterfly Women's Cotton Camisole,1,softline butterfly women s cotton camisole
2,Indian Economy: For UPSC Civil Services & Othe...,0,indian economy for upsc civil services other s...
3,Fujifilm X-T3 Mirrorless Digital Camera with X...,2,fujifilm x t mirrorless digital camera with xf...
4,"Barata Formal Ties For Men, Navy Blue Tie Form...",1,barata formal ties for men navy blue tie forma...


In [ ]:
df['text'].iloc[7]

"GTC Multi Use Clothes Organizer, Bookcase, Storage Cabinet, Wardrobe Closet 75 x 47 x 146 CM Cloth Rack ( IT N - STW013 ) Strong storage ability - wardrobe is made up of polypropylene sheet and steel rod which is not hazarded material for kids, women, and men also we can use this to store clothes, kids toys, books. Each cube can hold up to 8-10kg's approx and full size:75 x 47 x 146 cm strong and sturdy design- made from select steel, environmentally-friendly resin and plastic connectors. It is strong enough to handle whatever you throw at it. This portable storage closet will meet your long term storage needs. Dustproof and waterproof doors- each cube features a door which can keep your items sealed away from dust and humidity. Manufactured with durable steel and dirt-proof, easy-to-clean resin for easy maintenance and care. Quick and easy assembly -assembly is tool-less and takes mere minutes when following the installation instructions available on the image. We trust that you will

In [ ]:
df['clean_text'].iloc[7]

'gtc multi use clothes organizer bookcase storage cabinet wardrobe closet x x cm cloth rack it n stw strong storage ability wardrobe is made up of polypropylene sheet and steel rod which is not hazarded material for kids women and men also we can use this to store clothes kids toys books each cube can hold up to kg s approx and full size x x cm strong and sturdy design made from select steel environmentally friendly resin and plastic connectors it is strong enough to handle whatever you throw at it this portable storage closet will meet your long term storage needs dustproof and waterproof doors each cube features a door which can keep your items sealed away from dust and humidity manufactured with durable steel and dirt proof easy to clean resin for easy maintenance and care quick and easy assembly assembly is tool less and takes mere minutes when following the installation instructions available on the image we trust that you will be impressed with this aesthetically pleasing design 

In [ ]:
# data splitting
x_train, x_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size = 0.2, random_state = 42,stratify=df['label'])

In [ ]:
x_train.head()

,clean_text
11542,jockey women s cotton spaghetti top feel at ea...
4332,ansio vacuum compressed pp space saver storage...
7342,think and grow rich about the author born into...
8391,ccna data center dcict official cert guide e c...
6407,sathiyas sunrise cotton blend bath towel pack ...


# 2.b

## TD-IDF Method

In [ ]:
# remove stopwords and do the vectorization with TF-IDF on train and test data

list_stopwords = stopwords.words('english')

vectorizer = TfidfVectorizer(stop_words=list_stopwords)
train_tfidf = vectorizer.fit_transform(x_train)
test_tfidf = vectorizer.transform(x_test)

# convert to DataFrame
TFIDF_train = pd.DataFrame(train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
TFIDF_test = pd.DataFrame(test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
train_tfidf.shape

(10084, 37274)

In [ ]:
test_tfidf.shape

(2522, 37274)

> before tuning

In [ ]:
svm_model = svm.LinearSVC(random_state=42)
svm_model.fit(TFIDF_train, y_train)
svm_notuning = svm_model.predict(TFIDF_test)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(TFIDF_train, y_train)
rf_notuning = rf_model.predict(TFIDF_test)

In [ ]:
print('\n--------------------TF-IDF Before Tuning:--------------------\n')

print('\nClassification Report SVM:\n')
print(classification_report(y_test, svm_notuning, target_names = ['Electronics', 'Clothing & Accessories', 'Books', 'Household']))

print('\nClassification Report Random Forest:\n')
print(classification_report(y_test, rf_notuning, target_names = ['Electronics', 'Clothing & Accessories', 'Books', 'Household']))


--------------------TF-IDF Before Tuning:--------------------


Classification Report SVM:

                        precision    recall  f1-score   support

           Electronics       0.97      0.96      0.97       593
Clothing & Accessories       0.98      0.98      0.98       438
                 Books       0.97      0.95      0.96       523
             Household       0.96      0.98      0.97       968

              accuracy                           0.97      2522
             macro avg       0.97      0.97      0.97      2522
          weighted avg       0.97      0.97      0.97      2522


Classification Report Random Forest:

                        precision    recall  f1-score   support

           Electronics       0.95      0.94      0.95       593
Clothing & Accessories       0.98      0.95      0.96       438
                 Books       0.96      0.90      0.93       523
             Household       0.92      0.97      0.94       968

              accuracy         

Dari hasil di atas, dapat disimpulkan bahwa dengan menggunakan vectorization TF-IDF dengan algoritma SVM lebih unggul. Tidak hanya dikarenakan accuracy yang lebih tinggi, mencapai 97%, tetapi model SVM memiliki nilai precision-recall yang lebih tinggi pada setiap kelas prediksi, hal ini menunjukan bahwa kinerja model SVM dapat mengklasifikasikan jumlah prediksi benar dengan lebih baik. Lalu, hasil f1 score pada model SVM yang lebih tinggi juga menunjukkan stabilitas/keseimbangan dari model ini dalam memprediksi output.

In [ ]:
svm_model = svm.LinearSVC(random_state=42, max_iter = 10000)

# define the parameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'max_iter': [1000, 2000, 3000, 5000, 10000],
    'tol': [1e-4, 1e-3, 1e-2]
}

# search the best parameters
grid_search = GridSearchCV(svm_model, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(TFIDF_train, y_train)
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Parameters:", best_params)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
Best Parameters: {'C': 1, 'max_iter': 1000, 'penalty': 'l2', 'tol': 0.0001}


In [ ]:
svm_model = svm.LinearSVC(random_state=42, max_iter=1000, C=1, penalty="l2", tol=0.0001)
svm_model.fit(TFIDF_train, y_train)

test_svm = svm_model.predict(TFIDF_test) # predict with svm on our test data

> Random Forest

In [ ]:
rf = RandomForestClassifier(random_state= 42)

# define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'criterion': ['gini', 'entropy'],
}

# search the best parameters
grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=5, verbose=1)
grid_search.fit(TFIDF_train, y_train)
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Parameters:", best_params)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'n_estimators': 500}


In [ ]:
rf = RandomForestClassifier(random_state=42, criterion= 'gini', max_depth=None, max_features='log2', n_estimators=500)

In [ ]:
rf.fit(train_tfidf, y_train)
test_rf = rf.predict(test_tfidf) # predict with rf on our test data

In [ ]:
print('\n--------------------TF-IDF After Tuning:--------------------\n')

print('\nClassification Report SVM:\n')
print(classification_report(y_test, test_svm, target_names = ['Electronics', 'Clothing & Accessories', 'Books', 'Household']))

print('\n Classification Report Random Forest:\n')
print(classification_report(y_test, test_rf, target_names = ['Electronics', 'Clothing & Accessories', 'Books', 'Household']))


--------------------TF-IDF After Tuning:--------------------


Classification Report SVM:

                        precision    recall  f1-score   support

           Electronics       0.97      0.96      0.97       593
Clothing & Accessories       0.98      0.98      0.98       438
                 Books       0.97      0.95      0.96       523
             Household       0.96      0.98      0.97       968

              accuracy                           0.97      2522
             macro avg       0.97      0.97      0.97      2522
          weighted avg       0.97      0.97      0.97      2522


 Classification Report Random Forest:

                        precision    recall  f1-score   support

           Electronics       0.95      0.95      0.95       593
Clothing & Accessories       0.99      0.96      0.97       438
                 Books       0.97      0.90      0.93       523
             Household       0.92      0.98      0.95       968

              accuracy         

Dapat diketahui bahwa hasil setelah tuning juga menunjukkan kinerja model SVM lebih baik dibandingkan RF, yaitu dengan accuracy model setinggi 97%.
> model RF memang memiliki nilai precision yang lebih tinggi (99%) seperti pada kelas "Books", tetapi jika dilihat hasil recall yang hanya sebesar 90% maka ditemukan bahwa model RF kurang mengenali/melewatkan data pada kategori books yang menyebabkan nilai F1 score yang lebih rendah.

Secara umum, jika dilihat nilai precision, recall, dan F1 score, model SVM setelah tuning dapat mengklasifikasikan data dengan lebih seimbang dan efektif.

## Word2Vec

In [ ]:
token_train = [word_tokenize(i) for i in x_train]
token_test = [word_tokenize(i) for i in x_test]

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# remove stopwords from train and test data
filtered_train = [[word for word in sublist if word not in stop_words] for sublist in token_train]
filtered_test = [[word for word in sublist if word not in stop_words] for sublist in token_test]


In [ ]:
import gensim
from gensim.models import Word2Vec

# fit to skipgram model
model_skipgram = gensim.models.Word2Vec(filtered_train, min_count = 3, vector_size = 50, window = 5, sg=1)

In [ ]:
# get the words vector
def get_word_vector(model, word):
    try:
        return model.wv.get_vector(word)
    except KeyError:
        return None

In [ ]:
def generate_word_vectors(filtered_tokens, word_vector_model, vector_length=50):
    X_vectors = []

    # iterate through each list of filtered tokens in the dataset
    for tokens in filtered_tokens:
        vectorized_tokens = [get_word_vector(word_vector_model, word) for word in tokens]
        vectorized_tokens = [vector for vector in vectorized_tokens if vector is not None]

        # compute the avg vector
        if vectorized_tokens:
            average_vector = sum(vectorized_tokens) / len(vectorized_tokens)
            X_vectors.append(average_vector)
        else:
            # if there's no valid vectors, impute with zero vector as a placeholder
            X_vectors.append([0] * vector_length)

    return X_vectors

In [ ]:
# transform token in both train and test into vectors with skipgram model
x_train_skipgram = generate_word_vectors(filtered_train, model_skipgram)
x_test_skipgram = generate_word_vectors(filtered_test, model_skipgram)

In [ ]:
x_train_skipgram[0:2]

[array([-0.30402318,  0.19283417,  0.03803872, -0.6530323 ,  0.05904849,
        -0.11689132,  0.42930403,  0.34116518, -0.74322194, -0.5314646 ,
         0.13015212, -0.30071893, -0.02745783,  0.19396354, -0.34333628,
         0.32798246,  0.29628766,  0.16818918, -0.5398763 , -0.5073331 ,
        -0.12192332,  0.03315916,  0.3613829 , -0.22716221, -0.4248043 ,
         0.07425211, -0.10366955, -0.03237503, -0.1808582 ,  0.1456697 ,
        -0.40589768, -0.21496604, -0.2832404 ,  0.3543915 , -0.34663436,
         0.4347138 ,  0.07000694, -0.08190543,  0.05967326, -0.56738955,
         0.20207798, -0.31158137,  0.11942665,  0.37575522,  0.49710125,
         0.383491  ,  0.35992053, -0.49217024,  0.2984871 ,  0.33125547],
       dtype=float32),
 array([-0.13585031,  0.02326652, -0.22734728, -0.2621607 , -0.05637735,
        -0.11407407,  0.21013193,  0.1709887 , -0.6570419 , -0.18772905,
        -0.03013478, -0.37101194, -0.1088875 ,  0.1182986 , -0.38482648,
         0.01815462,  0.314

In [ ]:
# predict with svm and rf model
svm_model2 = svm.LinearSVC(random_state=42)
svm_model2.fit(x_train_skipgram, y_train)
svm_notuning2 = svm_model2.predict(x_test_skipgram)

rf_model2 = RandomForestClassifier(random_state=42)
rf_model2.fit(x_train_skipgram, y_train)
rf_notuning2 = rf_model2.predict(x_test_skipgram)

In [ ]:
print('\n--------------------Word2Vec Before Tuning:--------------------\n')

print('\nClassification Report SVM:\n')
print(classification_report(y_test, svm_notuning2, target_names = ['Electronics', 'Clothing & Accessories', 'Books', 'Household']))

print('\nClassification Report Random Forest:\n')
print(classification_report(y_test, rf_notuning2, target_names = ['Electronics', 'Clothing & Accessories', 'Books', 'Household']))


--------------------Word2Vec Before Tuning:--------------------


Classification Report SVM:

                        precision    recall  f1-score   support

           Electronics       0.97      0.91      0.94       593
Clothing & Accessories       0.96      0.98      0.97       438
                 Books       0.93      0.91      0.92       523
             Household       0.92      0.96      0.94       968

              accuracy                           0.94      2522
             macro avg       0.95      0.94      0.94      2522
          weighted avg       0.94      0.94      0.94      2522


Classification Report Random Forest:

                        precision    recall  f1-score   support

           Electronics       0.98      0.94      0.96       593
Clothing & Accessories       0.98      0.98      0.98       438
                 Books       0.95      0.95      0.95       523
             Household       0.94      0.97      0.96       968

              accuracy       

Jika dilihat pada hasil di atas, model RF memiiki kinerja lebih baik dibandingkan SVM jika menggunakan vectorization word2vec-skipgram. Bahkan hampir pada seluruh kelas output, model RF memiliki nilai precision, recall, dan F1 yang lebih tinggi.
> pada SVM terdapat kelas dengan nilai precision yang tinggi, tetapi memiliki nilai recall yang rendah -> kurang harmonis dalam memprediksi output, adanya data yang terlewat.

Secara umum, model RF memiliki kinerja yang lebih baik dan stabil.

In [ ]:
svm_model2 = svm.LinearSVC(random_state=42, max_iter=10000)

# define parameters grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'max_iter': [1000, 2000, 3000, 5000, 10000],
    'tol': [1e-4, 1e-3, 1e-2]
}

# search the best parameters
grid_search = GridSearchCV(svm_model2, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(x_train_skipgram, y_train)
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Parameters:", best_params)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
Best Parameters: {'C': 100, 'max_iter': 1000, 'penalty': 'l1', 'tol': 0.01}


In [ ]:
svm_model2 = svm.LinearSVC(random_state=42, max_iter=1000, C=100, penalty="l1", tol=0.001)
svm_model2.fit(x_train_skipgram, y_train)

# predict on test data
test_svm2 = svm_model2.predict(x_test_skipgram)

> Random Forest

In [ ]:
rf = RandomForestClassifier(random_state= 42)

# define parameters grid
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'criterion': ['gini', 'entropy'],
}

# search the best parameters
grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(x_train_skipgram, y_train)
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Parameters:", best_params)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 500}


In [ ]:
rf_model2 = RandomForestClassifier(random_state=42, criterion= 'gini', max_depth=None, max_features='sqrt', n_estimators=500)
rf_model2.fit(x_train_skipgram, y_train)

# predict on test data
test_rf2 = rf_model2.predict(x_test_skipgram)

In [ ]:
print('\n--------------------Word2Vec After Tuning:--------------------\n')

print('\nClassification Report SVM Word2Vec:\n')
print(classification_report(y_test, test_svm2, target_names = ['Electronics', 'Clothing & Accessories', 'Books', 'Household']))

print('\nClassification Random Forest Word2Vec:\n')
print(classification_report(y_test, test_rf2, target_names = ['Electronics', 'Clothing & Accessories', 'Books', 'Household']))


--------------------Word2Vec After Tuning:--------------------


Classification Report SVM Word2Vec:

                        precision    recall  f1-score   support

           Electronics       0.97      0.91      0.94       593
Clothing & Accessories       0.95      0.98      0.97       438
                 Books       0.93      0.91      0.92       523
             Household       0.93      0.96      0.94       968

              accuracy                           0.94      2522
             macro avg       0.94      0.94      0.94      2522
          weighted avg       0.94      0.94      0.94      2522


Classification Random Forest Word2Vec:

                        precision    recall  f1-score   support

           Electronics       0.98      0.94      0.96       593
Clothing & Accessories       0.98      0.98      0.98       438
                 Books       0.95      0.95      0.95       523
             Household       0.94      0.97      0.96       968

              accur

Untuk hasil setelah tuning, jika dilihat sekilas pada nilai  akurasi, kedua model tidak ada kenaikan signifikan dan tetap mempertahankan kinerja mereka, SVM -> 94% dan RF -> 96%, hasil yang sama didapatkan seperti sebelum dituning.
> Tetapi, jika dilihat lebih detail pada model SVM terdapat sedikit penurunan pada kategori 'Clothing & Accessories' di mana nilai precision sebelum tuning -> 96% menjadi 95% setelah di tuning. Menandakan ada lebih banyak data false positive dibandingkan sebelum tuning. Hal ini dapat terjadi karena adanya tuning penalty menggunakan 'l1' dan c = 100 yang mungkin akan meningkatkan kehati-hatian model dalam prediksi data dan menjadikan model lebih selektif, tetapi dapat menyebabkan adanya nilai true posistive yang terlewat dan menyebabkan precision turun.

# 2.c

# Conclusion:
> TF-IDF
 lebih cocok menggunakan algoritma SVM dikarenakan TF-IDF akan mengukur seberapa penting kata tersebut dengan mengukur jumlah munculnya -> data sparse. Lalu, SVM bekerja dengan mencari batas yang jelas antar kelas dengan fitur yang relevan di mana selaras dengan hasil dari TF-IDF untuk menemukan kata yang penting. Sehingga dari hasil komputasi sebelum dan sesudah tuning konsisten bahwa model SVM bekerja lebih baik dengan tingkat akurasi sebesar 97%.

 > Word2Vec - skipgram
 lebih kompatibel dengan algoritma RF, hal ini dikarenakan word2vec akan menyimpan dalam bentuk vektor yang berisi hubungan antar kata. Lalu, RF bekerja dengan membuat pohon keputusan dan dapat mengelola data dari hasil word2vec lebih maksimal dibandigkan dengan SVM karena RF akan menggunakan hubungan antar kata dalam vektor word2vec dan mengenali pola yang lebih kompleks, sehingga kombinasi keduanya dapat memberikan hasil yang lebih optimal, dengan tingkat akurasi sebesar 96%.
